In [2]:
import requests
import pandas as pd

dataframes = [pd.DataFrame() for _ in range(12)]


#base_url
base_url = "https://ckan0.cf.opendata.inter.prod-toronto.ca"

#query
query="/api/3/action/package_show"

url = base_url + query

params = {"id": "budget-operating-budget-program-summary-by-expenditure-category"}

package = requests.get(url, params=params).json()

# To get resource data:
for idx, resource in enumerate(package["result"]["resources"]):

    # To get metadata for non datastore_active resources:
    if not resource["datastore_active"]:
        url = base_url + "/api/3/action/resource_show?id=" + resource["id"]
        resource_metadata = requests.get(url).json()
        dURL = str(resource_metadata["result"]["url"])

        # Check if the URL contains years from 2014 to 2023
        if any(str(year) in resource_metadata["result"]["url"] for year in range(2014, 2024)):
            
            if (("preliminary" not in dURL) and ("recommended" not in dURL)):

                # TEST Print the resource URL
                print(idx, dURL)
                
                # try different sheet names:
                # Option 1 = sheet by order 0
                # Option 2 = Open Data
                # Option 3 = Year
                try:
                    df2 = pd.read_excel(resource_metadata["result"]["url"], sheet_name=0, header=0)
                    dataframes[idx] = df2
                    
                    if df2.empty:
                        df3 = pd.read_excel(resource_metadata["result"]["url"], sheet_name='Open Data', header=0)
                        dataframes[idx] = df3
                
                except:
                    for yr in range(2014, 2024):
                        try:
                            df = pd.read_excel(resource_metadata["result"]["url"], sheet_name=str(yr), header=0)
                            dataframes[idx] = df
                            break
                        except:
                            print(f"sheetname failed: {yr}")

0 https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/2c90a5d3-5598-4c02-abf2-169456c8f1f1/resource/a6f7a8e8-e497-4f77-9881-daba429ea981/download/approved-operating-budget-summary-2023.xlsx
1 https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/2c90a5d3-5598-4c02-abf2-169456c8f1f1/resource/9e5f9a63-fdeb-46e4-9f5f-8143038de56d/download/approved-operating-budget-summary-2022.xlsx
2 https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/2c90a5d3-5598-4c02-abf2-169456c8f1f1/resource/930502fa-87f1-4d71-8b71-4435df594b38/download/2021-open-data2.xlsx
sheetname failed: 2014
sheetname failed: 2015
sheetname failed: 2016
sheetname failed: 2017
sheetname failed: 2018
sheetname failed: 2019
sheetname failed: 2020
3 https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/2c90a5d3-5598-4c02-abf2-169456c8f1f1/resource/4e58558a-4773-4cd0-a16a-de481f25cb4d/download/2020-operating-open-data.xlsx
4 https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/2c90a5d3-5598-4c02-abf2-169456c8f1f1/resou

C:\Users\lukas\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [3]:
for df in dataframes:
    print(df.shape[0])
    print(df.columns)
    print("-"*20)

20127
Index([          'Program',           'Service',          'Activity',
         'Expense/Revenue',     'Category Name', 'Sub-Category Name',
         'Commitment item',                2023],
      dtype='object')
--------------------
19714
Index([          'Program',           'Service',          'Activity',
         'Expense/Revenue',     'Category Name', 'Sub-Category Name',
         'Commitment item',                2022],
      dtype='object')
--------------------
20816
Index([          'Program',           'Service',          'Activity',
         'Expense/Revenue',     'Category Name', 'Sub-Category Name',
         'Commitment item',                2021],
      dtype='object')
--------------------
17779
Index([          'Program',           'Service',          'Activity',
         'Expense/Revenue',     'Category Name', 'Sub-Category Name',
         'Commitment item',                2020],
      dtype='object')
--------------------
19428
Index([          'Program',           

In [4]:
budget2014_df = dataframes[10]
budget2015_df = dataframes[9]
budget2016_df = dataframes[7] 
budget2017_df = dataframes[6] 
budget2018_df = dataframes[5] 
budget2019_df = dataframes[4] 
budget2020_df = dataframes[3] 
budget2021_df = dataframes[2] 
budget2022_df = dataframes[1]
budget2023_df = dataframes[0]

#create new List to maintain original dataframes
df_list = [
    budget2014_df,
    budget2015_df,
    budget2016_df,
    budget2017_df,
    budget2018_df,
    budget2019_df,
    budget2020_df,
    budget2021_df,
    budget2022_df,
    budget2023_df,
]

In [5]:
#Renaming the last column using a loop
for i, df in enumerate(df_list):
    year = 2014 + i
    new_column_name = f'Budgeted amount {year}'
    df.rename(columns={year: new_column_name}, inplace=True)

In [6]:
budget2023_df.head(3)

,Program,Service,Activity,Expense/Revenue,Category Name,Sub-Category Name,Commitment item,Budgeted amount 2023
0,311 Toronto,311 Development,311 PMO,Expenses,Benefits,Benefits,Canada Pension Plan,63293.40
1,311 Toronto,311 Development,311 PMO,Expenses,Service And Rent,Telecommnuncation,Cellular Telephones,2830.00
2,311 Toronto,311 Development,311 PMO,Expenses,Benefits,Benefits,Comprehensive Medical,72595.32


In [11]:
#Changing the scientific notation 
format_sci_notation = lambda x: '{:.6e}'.format(float(x))
for df in df_list:
    last_col = df.columns[-1]
    df[last_col] = pd.to_numeric(df[last_col], errors='coerce')

In [14]:
budget2023_df.head(3)

,Program,Service,Activity,Expense/Revenue,Category Name,Sub-Category Name,Commitment item,Budgeted amount 2023
0,311 Toronto,311 Development,311 PMO,Expenses,Benefits,Benefits,Canada Pension Plan,63293.40
1,311 Toronto,311 Development,311 PMO,Expenses,Service And Rent,Telecommnuncation,Cellular Telephones,2830.00
2,311 Toronto,311 Development,311 PMO,Expenses,Benefits,Benefits,Comprehensive Medical,72595.32


In [24]:
#clean 2021 budget category "0"
budget2021_df = budget2021_df.loc[budget2021_df["Category Name"] != 0, :]

In [25]:
#Seperating the dataframes into expense and revenue dataframes
budget2014_df_exp = budget2014_df[budget2014_df['Expense/Revenue']=='Expenses']
budget2015_df_exp = budget2015_df[budget2015_df['Expense/Revenue']=='Expenses']
budget2016_df_exp = budget2016_df[budget2016_df['Expense/Revenue']=='Expenses']
budget2017_df_exp = budget2017_df[budget2017_df['Expense/Revenue']=='Expenses']
budget2018_df_exp = budget2018_df[budget2018_df['Expense/Revenue']=='Expenses']
budget2019_df_exp = budget2019_df[budget2019_df['Expense/Revenue']=='Expenses']
budget2020_df_exp = budget2020_df[budget2020_df['Expense/Revenue']=='Expenses']
budget2021_df_exp = budget2021_df[budget2021_df['Expense/Revenue']=='Expenses']
budget2022_df_exp = budget2022_df[budget2022_df['Expense/Revenue']=='Expenses']
budget2023_df_exp = budget2023_df[budget2023_df['Expense/Revenue']=='Expenses']

#Revenues
budget2014_df_rev = budget2014_df[budget2014_df['Expense/Revenue']=='Revenues']
budget2015_df_rev = budget2015_df[budget2015_df['Expense/Revenue']=='Revenues']
budget2016_df_rev = budget2016_df[budget2016_df['Expense/Revenue']=='Revenues']
budget2017_df_rev = budget2017_df[budget2017_df['Expense/Revenue']=='Revenues']
budget2018_df_rev = budget2018_df[budget2018_df['Expense/Revenue']=='Revenues']
budget2019_df_rev = budget2019_df[budget2019_df['Expense/Revenue']=='Revenues']
budget2020_df_rev = budget2020_df[budget2020_df['Expense/Revenue']=='Revenues']
budget2021_df_rev = budget2021_df[budget2021_df['Expense/Revenue']=='Revenues']
budget2022_df_rev = budget2022_df[budget2022_df['Expense/Revenue']=='Revenues']
budget2023_df_rev = budget2023_df[budget2023_df['Expense/Revenue']=='Revenues']

In [27]:
budget2021_df_rev.head(3)

,Program,Service,Activity,Expense/Revenue,Category Name,Sub-Category Name,Commitment item,Budgeted amount 2021
13,311 Toronto,311 Development,311 PMO,Revenues,Transfers From Capital,Transfers From Capital,Trans Fr Capital Fnd,-1928204.0
87,311 Toronto,311 Service Delivery,Not assigned,Revenues,Other Revenue,Inter-Divisional Recoveries,IDR-SSHA,-162966.0
88,311 Toronto,311 Service Delivery,Not assigned,Revenues,Other Revenue,Inter-Divisional Recoveries,IDR-Health,-169235.0
